# Connect Colab to a Chameleon server

This notebook describes how to connect Colab to a server running on Chameleon. This allows you to run experiments requiring bare metal access, storage, memory, GPU and compute that exceeds the abilities of Colab's hosted runtime, but with Colab's familiar interface (and notebooks stored in your Google Drive). It also allows you to easily go back and forth between the convenience of Colab's hosted runtime and Chameleon's greater capabilities, depending on the needs of your experiment.

## Provision the resource


### Check resource availability

This notebook will try to reserve a bare metal CentOS server with RTX6000 GPU on CHI@UC - pending availability. Before you begin, you should check the host calendar at [https://chi.uc.chameleoncloud.org/project/leases/calendar/host/](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/). In the "Node Type" dropdown, filter on `gpu_rtx_6000` and make sure some hosts are available.

### Chameleon configuration

You can change your Chameleon project name (if not using the one that is automatically configured in the JupyterHub environment) and the site on which to reserve resources (depending on availability) in the following cell.

In [ ]:
import chi, os

PROJECT_NAME = os.getenv('OS_PROJECT_NAME')
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)


If you need to change the details of the Chameleon server, e.g. use a different OS image, or a different node type depending on availability, you can do that in the following cell.

In [ ]:
chi.set("image", "CC-CentOS8-stream-CUDA11")
NODE_TYPE = "gpu_rtx_6000"

### Reservation

The following cell will create a reservation that begins now, and ends in 8 hours. You can modify the start and end date as needed.

In [ ]:
from chi import lease


res = []
lease.add_node_reservation(res, node_type=NODE_TYPE, count=1)
lease.add_fip_reservation(res, count=1)
start_date, end_date = lease.lease_duration(days=0, hours=8)

l = lease.create_lease(f"{os.getenv('USER')}-{NODE_TYPE}-centos", res, start_date=start_date, end_date=end_date)
l = lease.wait_for_active(l["id"])

### Provisioning resources

This cell provisions resources. It will take approximately 10 minutes. You can check on its status in the Chameleon web-based UI: [https://chi.uc.chameleoncloud.org/project/instances/](https://chi.uc.chameleoncloud.org/project/instances/), then come back here when it is in the READY state.

In [ ]:
from chi import server

reservation_id = lease.get_node_reservation(l["id"])
server.create_server(
    f"{os.getenv('USER')}-{NODE_TYPE}-centos", 
    reservation_id=reservation_id,
    image_name=chi.get("image")
)
server_id = server.get_server_id(f"{os.getenv('USER')}-{NODE_TYPE}-centos")
server.wait_for_active(server_id)

Associate an IP address with this server:

In [ ]:
reserved_fip = lease.get_reserved_floating_ips(l["id"])[0]
server.associate_floating_ip(server_id,reserved_fip)

and wait for it to come up:

In [ ]:
server.wait_for_tcp(reserved_fip, port=22)

## Install software

The following cells will install some basic packages in order to connect your Colab frontend to your Chameleon server. However, you may want to log in to your Chameleon server in order to access its terminal and install or configure packages outside of Colab.

To log in to the resource, use File > New > Terminal in the Chameleon JupyterHub environment, or your local terminal, and run:


In [ ]:
print("cc@" + reserved_fip)

Meanwhile, install Python3.8 and JupyterHub on your resource:

In [ ]:
from chi import ssh

node = ssh.Remote(reserved_fip)

In [ ]:
node.run('sudo dnf -y remove python3-pip')
node.run('sudo dnf makecache')
node.run('sudo dnf module -y install python38')
node.run('sudo dnf -y install python38-devel')
node.run('sudo alternatives --set python3 /usr/bin/python3.8')
node.run('sudo alternatives --set python /usr/bin/python3.8')
node.run('sudo python3 -m pip install --upgrade pip')

In [ ]:
node.run('sudo dnf install -q -y epel-release')
node.run('sudo dnf config-manager --add-repo https://developer.download.nvidia.com/compute/cuda/repos/rhel8/x86_64/cuda-rhel8.repo')
node.run('sudo dnf install -y libcudnn8-8.3.3.40-1.cuda11.5')

#### Install Python packages

Before starting your Jupyter instance, you may want to install some Python packages.

The following cells will install the *same* version of some key deep learning packages as are installed on Colab (as of November 2022), for maximum cross-compatibility. 

In [ ]:
node.run('sudo dnf -y install gdal-devel libpq-devel libsndfile-devel cairo-devel openmpi-devel')
node.run('python3 -m pip install --user Cython==0.29.32')
node.run('wget https://raw.githubusercontent.com/teaching-on-testbeds/colab/main/requirements_chameleon_dl.txt -O requirements_chameleon_dl.txt')
node.run('wget https://raw.githubusercontent.com/teaching-on-testbeds/colab/main/requirements_chameleon.txt -O requirements_chameleon.txt')
node.run('python3 -m pip install --user -r requirements_chameleon_dl.txt --extra-index-url https://download.pytorch.org/whl/cu113 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html')

If you need additional packages, you can install them in a similar manner.

Test your installation - make sure Tensorflow, Pytorch, and JAX can all see the GPU:

In [ ]:
node.run('python3 -c \'import tensorflow as tf; print(tf.config.list_physical_devices("GPU"))\'')
# should say: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

In [ ]:
node.run('python3 -c \'import torch; print(torch.cuda.get_device_name(0))\'')
# should say: Quadro RTX 6000

In [ ]:
node.run('python3 -c \'import jax; print(jax.devices())\'')
# should say: StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)

### Set up Jupyter on server

Install `jupyter_http_over_ws`, which is required in order to connect Colab to this Jupyter instance:

In [ ]:
node.run('python3 -m pip install --user  jupyter-core jupyter-client jupyter_http_over_ws traitlets -U --force-reinstall')

And, active `jupyter_http_over_ws`:

In [ ]:
node.run('jupyter serverextension enable --py jupyter_http_over_ws')

## Connect Colab to the server

In a **local terminal on your own laptop**, run

In [ ]:
print('ssh -L 127.0.0.1:8888:127.0.0.1:8888 cc@' + reserved_fip) 

to set up a tunnel to the Jupyter server. If your Chameleon key is not in the default location, you should also specify the path to your key as an argument, using `-i`.  Leave this SSH session open.

Then, run the following cell, which will run a command that does not terminate: 

In [ ]:
node.run("jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0")

In the output of the cell above, look for a URL in this format:
    
```
http://localhost:8888/?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

Copy this URL - you will need it in the next step.

Now, you can open Colab in a browser. Click on the drop-down menu for "Connect" in the top right and select "Connect to a local runtime". Paste the URL you copied earlier into the space and click "Connect". Your notebook should now be running on your Colab host (you can put `!hostname` in a cell and run it to verify!)


## Release resources

If you finish with your experimentation before your lease expires,release your resources and tear down your environment by running the following (commented out to prevent accidental deletions).

This section is designed to work as a "standalone" portion - you can come back to this notebook, ignore the top part, and just run this section to delete your reasources.

In [ ]:
# setup environment - if you made any changes in the top part, make the same changes here
import chi, os
from chi import lease, server

PROJECT_NAME = os.getenv('OS_PROJECT_NAME')
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)

NODE_TYPE = "gpu_rtx_6000"
lease = chi.lease.get_lease(f"{os.getenv('USER')}-{NODE_TYPE}-centos")

In [ ]:
DELETE = False
# un-comment this next line to delete resources
# DELETE = True  

if DELETE:
    # delete server
    server_id = chi.server.get_server_id(f"{os.getenv('USER')}-{NODE_TYPE}-centos")
    chi.server.delete_server(server_id)

    # release floating IP
    reserved_fip =  chi.lease.get_reserved_floating_ips(lease["id"])[0]
    ip_info = chi.network.get_floating_ip(reserved_fip)
    chi.neutron().delete_floatingip(ip_info["id"])

    # delete lease
    chi.lease.delete_lease(lease["id"])
